Aother advanced generator feature is the throw method for re-raising Exception instances within generator functions.

* The way throw works is simple: When the method is called, the next occurence of a yield expression re-raises the provided Exception instance after its output is received instead of continuing normally.

In [4]:
class MyError(Exception):
    pass

def my_generator():
    yield 1
    yield 2
    yield 3
    
it = my_generator()
print(next(it)) #Yield 1
print(next(it)) #Yield 2
print(it.throw(MyError('test error')))

1
2


MyError: test error

When you call throw, the generator function may catch the injected exception with a standard try/except compound statement that surrounds the last yield expression that was executed

In [5]:
def my_generator():
    yield 1
    try:
        yield 2
    except MyError:
        print('Got MyError!')
    else:
        yield 3
    yield 4
    
it = my_generator()
print(next(it))
print(next(it))
print(it.throw(MyError('test error')))

1
2
Got MyError!
4


In [18]:
class Reset(Exception):
    pass

def timer(period):
    current = period
    while current:
        current -= 1
        try:
            yield current
        except Reset:
            current = period

def check_for_reset(current,timer=[3]):
    if current == 1 and timer[0] != 0:
        timer[0] -= 1
        return True
    return False

def announce(remaining):
    print(f'{remaining} ticks remaining')
    
def run():
    it = timer(4)
    current = 99
    while True:
        try:
            if check_for_reset(current):
                current = it.throw(Reset())
            else:
                current = next(it)
        except StopIteration:
            break
        else:
            announce(current)
run()

3 ticks remaining
2 ticks remaining
1 ticks remaining
3 ticks remaining
2 ticks remaining
1 ticks remaining
3 ticks remaining
2 ticks remaining
1 ticks remaining
3 ticks remaining
2 ticks remaining
1 ticks remaining
0 ticks remaining


In [20]:
#A simpler approach to implementing htis functionality is to define a stateful closure
#using an iterable container object
class Timer:
    def __init__(self, period):
        self.current = period
        self.period = period
    
    def reset(self):
        self.current = self.period
    
    def __iter__(self):
        while self.current:
            self.current -= 1
            yield self.current

def check_for_reset(current,timer=[3]):
    if current == 1 and timer[0] != 0:
        timer[0] -= 1
        return True
    return False

def run():
    timer = Timer(4)
    for current in timer:
        if check_for_reset(current):
            timer.reset()
        announce(current)
        
run()

3 ticks remaining
2 ticks remaining
1 ticks remaining
3 ticks remaining
2 ticks remaining
1 ticks remaining
3 ticks remaining
2 ticks remaining
1 ticks remaining
3 ticks remaining
2 ticks remaining
1 ticks remaining
0 ticks remaining
